# 1. Information about the submission

## 1.1 Name and number of the assignment 

HW №1
Argument Mining Competition (RuArg-2022)


## 1.2 Student name

Долгодворова Мария Александровна

## 1.3 Codalab user ID

knapweedss 

## 1.4 Additional comments

# 2. Technical Report

FastText улучшает работу модели и превосходит результат TF-IDF (в рамках этого задания - на 0.3). 
Кроме того, для модели основанной на FastText важна предобработка данных и правильно выстроенные параметры

## 2.1 Methodology 

В качестве бейзлайна был выбран RuAtg_baseline, поскольку, как мне кажется, это подходит для задачи классификации
- предобработка данных (пунктуация, лемматизация)
- Tf-IDF
- FastText 

## 2.2 Discussion of results

Мне кажется, 0.4854 не совсем плохой результат в рамках этого задания, однако, хорошим его назвать тоже сложно

Вероятно, можно попробовать другой подход к решению этой задачи 

# 3. Code

## 3.1 Requirements

In [ ]:
from tqdm import tqdm
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
! pip install fasttext
import fasttext
import csv
from tqdm.notebook import tqdm
import pandas as pd

## 3.2 Download the data

In [2]:
!wget -O train.tsv https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/train.tsv
!wget -O val_empty.tsv https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/val_empty.tsv
!wget -O test-no_labels.tsv https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/test-no_labels.tsv

--2022-11-13 19:39:23--  https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1539551 (1.5M) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>]   1.47M  --.-KB/s    in 0.008s  

2022-11-13 19:39:23 (183 MB/s) - ‘train.tsv’ saved [1539551/1539551]

--2022-11-13 19:39:24--  https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/val_empty.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308303 (301K) [text/pla

In [23]:
train_df = pd.read_csv("train.tsv", sep="\t")
val_df = pd.read_csv("val_empty.tsv", sep="\t")
test_df = pd.read_csv("test-no_labels.tsv", sep="\t")

In [5]:
val_df.tail(3)

,text_id,text
1428,34023,"[USER], привожу, ибо они часто говорят и пишут..."
1429,34026,"Про насильную вакцинацию говорили только ВОЗ, ..."
1430,34036,"Вакцину нашу все хотят, потому что ни у кого т..."


## 3.3 Preprocessing 

In [26]:
train_df.tail(2)

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
6715,34037,"А потом будут говорить про ""российскую медицин...",__class__0,__class__0,__class__0,__class__0,__class__3,__class__2
6716,34039,"Вакцина не предотвращает заболевание, она прос...",__class__0,__class__0,__class__0,__class__0,__class__3,__class__3


In [28]:
! pip install pymorphy2
import re

from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return ' '.join(tokens)
    return ' '.join(tokens)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
train_df['text'] = train_df.text.apply(lemmatize)

In [30]:
test_df['text'] = test_df.text.apply(lemmatize)

In [31]:
val_df['text'] = val_df.text.apply(lemmatize)

## 3.4 Training 

In [32]:
res_data = test_df.copy()

label = '__class__'

def categorizer(data, class_name):
    # Select required columns
    data = data[['text_id', 'text', f'{class_name}_stance', f'{class_name}_argument']]

    # Set your model output as categorical and save in new label col
    data['stance_label'] = pd.Categorical(data[f'{class_name}_stance'])
    data['argument_label'] = pd.Categorical(data[f'{class_name}_argument'])

    # Transform your output to numeric
    data[f'{class_name}_stance'] = data['stance_label'].cat.codes
    data[f'{class_name}_argument'] = data['argument_label'].cat.codes
    return data

for class_name in tqdm(["quarantine", "vaccines",  "masks"]):
    tmp_data = categorizer(train_df, class_name=class_name)
    train_data = tmp_data[['text', f'{class_name}_stance', f'{class_name}_argument']]
    for case in ['stance', 'argument']:
        train_df[f'{class_name}_{case}'] = label + train_data[f'{class_name}_{case}'].astype(str)
stopwords = set(nltk_stopwords.words("russian"))
nltk.download('stopwords')  
text_transformer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 2), lowercase=False, max_features=150000)
X_train_text = text_transformer.fit_transform(train_df['text'])
X_test_text = text_transformer.transform(test_df['text'])

ros = RandomOverSampler(random_state=42)

for class_name in tqdm(["quarantine", "vaccines",  "masks"]):
    for case in ['stance', 'argument']:
        x_ros, y_ros = ros.fit_resample(X_train_text, train_df[[f'{class_name}_{case}']])
        logit = LogisticRegression(C=6e1, solver='newton-cg', multi_class='multinomial', random_state=17, n_jobs=4)
        logit.fit(x_ros, y_ros)
        res_data[f'{class_name}_{case}'] = logit.predict(X_test_text)

# postprocessing
for col in res_data.columns:
  if 'text' not in col:
    #res_data[col] = res_data[col].astype(int) - 1
    res_data[col] = res_data[col].apply(lambda x : x[-1]).astype(int) - 1

for claim in ["quarantine", "vaccines",  "masks"]:
  res_data.loc[(res_data[f'{claim}_stance'] == -1) & (res_data[f'{claim}_argument'] != -1), [f'{claim}_argument']] = -1
  res_data.loc[(res_data[f'{claim}_stance'] > -1) & (res_data[f'{claim}_argument'] == -1), [f'{claim}_argument']] = 0

res_data.to_csv('logreg2_6.tsv', index = False, sep='\t')
!zip logreg2_6.zip logreg2_6.tsv

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

updating: logreg2_6.tsv (deflated 77%)


In [33]:
X_train_text.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
res_data = test_df.copy()

In [35]:
ros = RandomOverSampler(random_state=42)
for class_name in tqdm(["quarantine", "vaccines",  "masks"]):
    for case in ['stance', 'argument']:     
        variablex_ros_qs, y_ros_qs = ros.fit_resample(X_train_text, train_df[[f'{class_name}_{case}']])


rf=RandomForestClassifier(max_depth=7,n_estimators=300)

res_data_rf = test_df.copy()
ros = RandomOverSampler(random_state=42)

for class_name in tqdm(["quarantine", "vaccines",  "masks"]):
    for case in ['stance', 'argument']:
        x_ros, y_ros = ros.fit_resample(X_train_text, train_df[[f'{class_name}_{case}']])
        rf.fit(x_ros, y_ros)
        res_data_rf[f'{class_name}_{case}'] = rf.predict(X_test_text)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel(

In [36]:
res_data_rf

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,17059,каникулы дать почва распостранение вирус нужно...,__class__0,__class__0,__class__2,__class__2,__class__0,__class__0
1,17072,думать спокойно посидеть небольшой компания по...,__class__0,__class__0,__class__3,__class__3,__class__0,__class__0
2,17077,китай болезнь гулять декабрь просто последний ...,__class__0,__class__0,__class__2,__class__2,__class__0,__class__0
3,17082,переболеть бессимптомный добрый треть населени...,__class__0,__class__0,__class__2,__class__3,__class__0,__class__3
4,17089,а подготовка подразумевать себя массовый скопл...,__class__0,__class__0,__class__3,__class__2,__class__0,__class__0
...,...,...,...,...,...,...,...,...
1397,33948,интересный штука точно уверенный свой здоровье...,__class__0,__class__0,__class__0,__class__0,__class__3,__class__3
1398,33957,если обещать вариант вакцина скорый время нача...,__class__0,__class__0,__class__0,__class__0,__class__3,__class__2
1399,33960,скоро вакцина зараза стать доступный сделать п...,__class__0,__class__0,__class__0,__class__0,__class__3,__class__0
1400,33993,трудность конец год каждый желать получить воз...,__class__0,__class__0,__class__0,__class__0,__class__3,__class__2


In [37]:
# postprocessing
for col in res_data_rf.columns:
  if 'text' not in col:
    #res_data[col] = res_data[col].astype(int) - 1
    res_data_rf[col] = res_data_rf[col].apply(lambda x : x[-1]).astype(int) - 1

for claim in ["quarantine", "vaccines",  "masks"]:
  res_data_rf.loc[(res_data_rf[f'{claim}_stance'] == -1) & (res_data_rf[f'{claim}_argument'] != -1), [f'{claim}_argument']] = -1
  res_data_rf.loc[(res_data_rf[f'{claim}_stance'] > -1) & (res_data_rf[f'{claim}_argument'] == -1), [f'{claim}_argument']] = 0
res_data_rf

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,17059,каникулы дать почва распостранение вирус нужно...,-1,-1,1,1,-1,-1
1,17072,думать спокойно посидеть небольшой компания по...,-1,-1,2,2,-1,-1
2,17077,китай болезнь гулять декабрь просто последний ...,-1,-1,1,1,-1,-1
3,17082,переболеть бессимптомный добрый треть населени...,-1,-1,1,2,-1,-1
4,17089,а подготовка подразумевать себя массовый скопл...,-1,-1,2,1,-1,-1
...,...,...,...,...,...,...,...,...
1397,33948,интересный штука точно уверенный свой здоровье...,-1,-1,-1,-1,2,2
1398,33957,если обещать вариант вакцина скорый время нача...,-1,-1,-1,-1,2,1
1399,33960,скоро вакцина зараза стать доступный сделать п...,-1,-1,-1,-1,2,0
1400,33993,трудность конец год каждый желать получить воз...,-1,-1,-1,-1,2,1


In [38]:
def categorizer(data, class_name):
    # Select required columns
    data = data[['text_id', 'text', f'{class_name}_stance', f'{class_name}_argument']]

    # Set your model output as categorical and save in new label col
    data['stance_label'] = pd.Categorical(data[f'{class_name}_stance'])
    data['argument_label'] = pd.Categorical(data[f'{class_name}_argument'])

    # Transform your output to numeric
    data[f'{class_name}_stance'] = data['stance_label'].cat.codes
    data[f'{class_name}_argument'] = data['argument_label'].cat.codes
    return data

In [39]:
tmp_data = categorizer(train_df, class_name='quarantine')
tmp_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

,text_id,text,quarantine_stance,quarantine_argument,stance_label,argument_label
0,17024,согласно предписание роспотребнадзор транзитны...,2,2,__class__2,__class__2
1,17025,о несоблюдение карантинный мера контактный лиц...,2,2,__class__2,__class__2
2,17027,читать книжка карантин мозг реанимировать,2,2,__class__2,__class__2
3,17030,идти почитай инст наш городской пабликовый каж...,2,2,__class__2,__class__2
4,17031,всё контактный лицо который обозначить отправи...,2,2,__class__2,__class__2
...,...,...,...,...,...,...
6712,34033,заботить врач который вынудить такой запаковыв...,0,0,__class__0,__class__0
6713,34034,жрать жрать вирус народ вакцина мы прийти,0,0,__class__0,__class__0
6714,34035,я вопрос волновать ещё убить этот вакцина кром...,0,0,__class__0,__class__0
6715,34037,а быть говорить российский медицина заразить м...,0,0,__class__0,__class__0


In [40]:
res_data = test_df.copy()

label = '__class__'
for class_name in tqdm(["quarantine", "vaccines",  "masks"]):
  tmp_data = categorizer(train_df, class_name=class_name)
  train_data = tmp_data[['text', f'{class_name}_stance', f'{class_name}_argument']]
  for case in ['stance', 'argument']:
    train_df[f'{class_name}_{case}'] = label + train_df[f'{class_name}_{case}'].astype(str)
    train_df[[f'{class_name}_{case}', 'text']].to_csv('train.txt', index = False, header = None, sep = ' ',
                                                     quoting = csv.QUOTE_NONE, 
                                                     quotechar = "", escapechar = " ")

    ft_model = fasttext.train_supervised('train.txt', label=label, epoch=30, lr=0.1, ws = 5, thread=12, loss='softmax', wordNgrams=2)
    texts = test_df['text'].to_list()
    preds = ft_model.predict(texts)[0]
    res_data[f'{class_name}_{case}'] = [pred[0].replace(label,'') for pred in preds]

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [41]:
res_data = test_df.copy()

label = '__class__'
for class_name in tqdm(["quarantine", "vaccines",  "masks"]):
  tmp_data = categorizer(train_df, class_name=class_name)
  train_data = tmp_data[['text', f'{class_name}_stance', f'{class_name}_argument']]
  for case in ['stance', 'argument']:
    train_df[f'{class_name}_{case}'] = label + train_df[f'{class_name}_{case}'].astype(str)
    train_df[[f'{class_name}_{case}', 'text']].to_csv('train.txt', index = False, header = None, sep = ' ',
                                                     quoting = csv.QUOTE_NONE, 
                                                     quotechar = "", escapechar = " ")

    ft_model = fasttext.train_supervised('train.txt', label=label, epoch=30, lr=0.1, ws = 5, thread=12, loss='softmax', wordNgrams=2)
    texts = test_df['text'].to_list()
    preds = ft_model.predict(texts)[0]
    res_data[f'{class_name}_{case}'] = [pred[0].replace(label,'') for pred in preds]


from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(max_depth=15,n_estimators=300)



  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [42]:
# postprocessing
for col in res_data.columns:
  if 'text' not in col:
    res_data[col] = res_data[col].astype(int) - 1

for claim in ["quarantine", "vaccines",  "masks"]:
  res_data.loc[(res_data[f'{claim}_stance'] == -1) & (res_data[f'{claim}_argument'] != -1), [f'{claim}_argument']] = -1
  res_data.loc[(res_data[f'{claim}_stance'] > -1) & (res_data[f'{claim}_argument'] == -1), [f'{claim}_argument']] = 0

res_data

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,17059,каникулы дать почва распостранение вирус нужно...,-1,-1,2,1,-1,-1
1,17072,думать спокойно посидеть небольшой компания по...,-1,-1,1,1,-1,-1
2,17077,китай болезнь гулять декабрь просто последний ...,-1,-1,2,1,-1,-1
3,17082,переболеть бессимптомный добрый треть населени...,-1,-1,2,1,-1,-1
4,17089,а подготовка подразумевать себя массовый скопл...,-1,-1,1,1,-1,-1
...,...,...,...,...,...,...,...,...
1397,33948,интересный штука точно уверенный свой здоровье...,-1,-1,-1,-1,2,1
1398,33957,если обещать вариант вакцина скорый время нача...,-1,-1,-1,-1,2,1
1399,33960,скоро вакцина зараза стать доступный сделать п...,2,1,-1,-1,2,1
1400,33993,трудность конец год каждый желать получить воз...,-1,-1,-1,-1,2,1


In [43]:
res_data.to_csv('Goutput.tsv', index = False, sep='\t')

In [44]:
!zip Goutput.zip Goutput.tsv

  adding: Goutput.tsv (deflated 77%)


## 3.4 Inference

- Изначально я пробовала только TF-IDF, Fasttext улучшил результат (но это был бейзлайн). В качестве модификации я увеличила количество n_estimators и max_depth, а также лемматизировала датафрейм - поскольку фасттекст учитывает N-граммы символов

- В результате качество незначительно, но возросло (0.46 - > 0.48)